# Claude-Anthropic with Conversation Chain

This document provides an overview of the `ClaudeSession` Python class designed for conversational interactions using Amazon Bedrock's Claude model and LangChain's ConversationChain.

## Overview

The `ClaudeSession` class facilitates a session-based conversation with Claude, a language model provided by Amazon Bedrock. It leverages LangChain's `ConversationChain` and `ConversationBufferMemory` to maintain context throughout the conversation, allowing for more natural and coherent dialogue.

In [ ]:
!pip install boto3 langchain


# Creating Claude Session

The `ask_question` method allows users to send questions to the model. The first call to this method sets the context for the conversation, and subsequent calls build upon this context:

In [ ]:
import json
import os
import sys
import boto3
from langchain.llms import Bedrock


os.environ["AWS_ACCESS_KEY_ID"]="access_key_id_here"
os.environ["AWS_SECRET_ACCESS_KEY"]="secret_access_key_here"

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

class ClaudeSession:
    def __init__(self):
        boto3_bedrock = boto3.client(
                        service_name="bedrock-runtime",
                        region_name="us-west-2",
                        aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
                        aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
                    )
        self.llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock)
        self.conversation = ConversationChain(
            llm=self.llm, verbose=True, memory=ConversationBufferMemory()
        )

    def ask_question(self, question):
      return self.conversation.predict(input=question)


# Data import
Currently I am using a json file for the import here to return the data

In [ ]:
import json

def read_file():
  file_path = "evidenceText.json"
  try:
      with open(file_path, 'r') as file:
          data = json.load(file)
          summaries = [item['summary'] for item in data if 'summary' in item]
          evidences = [item['evidence'] for item in data if 'evidence' in item]
          concatenated_summaries = "; ".join(summaries)
          concatenated_evidences = "; ".join(evidences)
          processed_data = {
              'summary': concatenated_summaries,
              'evidence': concatenated_evidences
          }
          return processed_data
  except FileNotFoundError:
      print(f"The file at path {file_path} does not exist.")
  except json.JSONDecodeError:
      print(f"The file at path {file_path} is not a valid JSON file.")

data = read_file()


# Asking questions

In [ ]:
my_session = ClaudeSession()

# Send the context in first
initial_context_response = my_session.ask_question(f"Here is a json file with the evidence information, I will be asking questions based on it \
                                                     in the next prompt(s): {data}")

# Ask questions based on conversation history
answer1 = my_session.ask_question("What are the various city names?")
print(answer1)
answer2 = my_session.ask_question("What tools are used?")
print(answer2)

